In [193]:
import os
import sys


sys.path.append(os.path.abspath(os.path.join("..")))

import numpy as np

# Local application imports
from utils.pricing_market_logic_multiproduct import (
    get_monopoly_prices,
    get_nash_prices,
    get_profits,
    get_quantities,
)

from sklearn.preprocessing import normalize

## 2.1 Economic Environment

The demand function follows the specification in Calvano et al. (2020b):

$$
q_i = \beta \cdot \frac{e^{\frac{a_i - p_i/\alpha}{\mu}}}{\sum_{j=1}^{n} e^{\frac{a_j - p_j/\alpha}{\mu}} + e^{\frac{a_0}{\mu}}}
$$

Where:

* $a_1, a_2, \ldots, a_n$: product-specific attributes (differentiation).
* $a_0$: baseline demand intercept.
* $\alpha, \beta$: scaling parameters.

  * $\alpha \in \{1, 3.2, 10\}$: affects currency unit; LLMs may not be unit-invariant.
  * $\beta = 100$: quantity scaling factor; chosen for LLM interpretability.
* $a_i = 2$, $a_0 = 0$, $\mu = 0.25$: all from Calvano et al. (2020b).

The profit function for firm $i$ is:

$$
\pi_i = (p_i - \alpha c_i) \cdot q_i
$$

with marginal cost $c_i = 1$ (following Calvano et al., 2020b).

## Functions


In [194]:
def compute_a_i(shares, prices, mu, alpha):
    """
    Computes the a_i parameters for a logit demand system given observed market shares and prices.

    Parameters:
    - shares: list or array of market shares for each firm (length n)
    - prices: list or array of baseline prices for each firm (length n)
    - mu: scale parameter (e.g., 0.25)
    - alpha: price scaling parameter (e.g., 1, 3.2, or 10)

    Returns:
    - a_i_values: list of computed a_i parameters for each firm
    """
    shares = np.array(shares)
    prices = np.array(prices)

    # s_0 = 1 - np.sum(shares)  # Outside option share

    # if s_0 <= 0:
    #     raise ValueError("Sum of market shares exceeds 1. Check your input data.")

    log_ratio = np.log(shares / 0.001)  # NOTE! CHECK AFTERWARDS
    a_i_values = mu * log_ratio + prices / alpha

    return a_i_values


def simulate_market_with_shares(shares, prices, alpha=1, beta=100, mu=0.25, a0=0, ci=1):
    """
    Computes monopoly and Nash equilibrium prices, quantities, profits, and updated market shares
    using the exact structure and functions from user's existing code.

    Returns:
    Dictionary with prices, quantities, market shares, and profits for both monopoly and Nash scenarios.
    """

    n_firms = len(shares)
    # # #Normalize shares to ensure they sum to 1
    shares_norm = normalize(np.array(shares).reshape(1, -1), norm="l1")[0]
    # print("Normalized Shares:", shares_norm)
    # print(1== sum(shares_norm))

    # Compute a_i values
    a_i_values = compute_a_i(shares_norm, prices, mu, alpha)

    a_vector = a_i_values
    alpha_vector = tuple([alpha for _ in range(n_firms)])
    c_vector = tuple([ci for _ in range(n_firms)])
    sigma = 0
    group_idxs = tuple([i for i in range(1, n_firms + 1)])

    # Monopoly Scenario
    monopoly_prices = get_monopoly_prices(
        a0=a0,
        a=a_vector,
        mu=mu,
        alpha=alpha_vector,
        c=c_vector,
        multiplier=beta,
        sigma=sigma,
        group_idxs=group_idxs,
    )
    monopoly_q = get_quantities(
        p=tuple(monopoly_prices),
        a0=a0,
        a=a_vector,
        mu=mu,
        alpha=alpha_vector,
        multiplier=beta,
        sigma=sigma,
        group_idxs=group_idxs,
    )
    monopoly_profits = get_profits(
        p=tuple(monopoly_prices),
        c=c_vector,
        a0=a0,
        a=a_vector,
        mu=mu,
        alpha=alpha_vector,
        multiplier=beta,
        sigma=sigma,
        group_idxs=group_idxs,
    )

    # Compute market shares
    total_quantity_monopoly = sum(monopoly_q)
    monopoly_market_shares = [q / total_quantity_monopoly for q in monopoly_q]

    # Nash Scenario
    nash_prices = get_nash_prices(
        a0=a0,
        a=a_vector,
        mu=mu,
        alpha=alpha_vector,
        multiplier=beta,
        sigma=sigma,
        group_idxs=group_idxs,
        c=c_vector,
    )
    nash_q = get_quantities(
        p=tuple(nash_prices),
        a0=a0,
        a=a_vector,
        mu=mu,
        alpha=alpha_vector,
        multiplier=beta,
        sigma=sigma,
        group_idxs=group_idxs,
    )
    nash_profits = get_profits(
        p=tuple(nash_prices),
        c=c_vector,
        a0=a0,
        a=a_vector,
        mu=mu,
        alpha=alpha_vector,
        multiplier=beta,
        sigma=sigma,
        group_idxs=group_idxs,
    )

    total_quantity_nash = sum(nash_q)
    nash_market_shares = [q / total_quantity_nash for q in nash_q]

    # Return results
    return {
        "monopoly_prices": monopoly_prices,
        "monopoly_quantities": monopoly_q,
        "monopoly_market_shares": monopoly_market_shares,
        "monopoly_profits": monopoly_profits,
        "nash_prices": nash_prices,
        "nash_quantities": nash_q,
        "nash_market_shares": nash_market_shares,
        "nash_profits": nash_profits,
    }

In [195]:
import polars as pl
from datetime import datetime
from pathlib import Path


def load_retail_data(
    file_path: str = "../../data/113176-V1/data/Prices/",
    file: str = "FuelWatchRetail-*.csv",
    start_date: datetime.date = datetime(2009, 1, 1),
    end_date: datetime.date = None,
) -> pl.DataFrame:
    """
    Load and concatenate retail price data from one or multiple CSV files.

    Parameters
    ----------
    file_path : str, optional
        Directory containing the retail price CSV files. Defaults to "../../data/113176-V1/data/Prices/".
    file : str, optional
        Glob pattern for the retail price files. Defaults to "FuelWatchRetail-*.csv".
    start_date : datetime.date, optional
        Start date for filtering the data. If None, no lower bound is applied.
    end_date : datetime.date, optional
        End date for filtering the data. If None, no upper bound is applied.

    Returns
    -------
    pl.DataFrame
        A Polars DataFrame containing the concatenated and filtered retail price data, sorted by 'PUBLISH_DATE'.
    """
    queries = []
    file_path = Path(file_path)

    start_condition = (
        pl.col("PUBLISH_DATE") >= start_date if start_date else pl.lit(True)
    )
    end_condition = pl.col("PUBLISH_DATE") <= end_date if end_date else pl.lit(True)

    for file in os.listdir(file_path):
        file = file_path / file
        q = pl.scan_csv(file, try_parse_dates=True).filter(
            start_condition
            & end_condition
            &
            # Filter data such that it replicates Byrne 2019 as in `MakeData1_RawPrices.do`
            (pl.col("PRODUCT_DESCRIPTION") == "ULP")  # Only unleaded fuels
            & (pl.col("POSTCODE") < 6200)  # Only Perth
            & (pl.col("ADDRESS") != "XXXXX")  # Must have an address
            & (
                pl.col("PUBLISH_DATE").is_between(  # Between Jan 2001-2015
                    datetime(2001, 1, 1), datetime(2015, 1, 1)
                )
            )
        )
        queries.append(q)

    dataframes = pl.collect_all(queries)
    return pl.concat(dataframes, how="diagonal_relaxed").sort("PUBLISH_DATE")

In [196]:
def load_tgp_data(
    file_path: str = "../../data/113176-V1/data/TGP/tgpmin.csv",
    start_date: datetime.date = None,
    end_date: datetime.date = None,
) -> pl.DataFrame:
    """
    Load and prepare TGP (Terminal Gate Price) data from a CSV file.

    Parameters
    ----------
    file_path : str, optional
        Path to the TGP CSV file. Defaults to "../../data/113176-V1/data/TGP/tgpmin.csv".
    start_date : datetime.date, optional
        Start date for filtering the data. If None, no lower bound is applied.
    end_date : datetime.date, optional
        End date for filtering the data. If None, no upper bound is applied.

    Returns
    -------
    pl.DataFrame
        A Polars DataFrame containing the filtered and sorted TGP data with the 'date' column parsed as dates.
    """
    # Create filter conditions
    start_condition = pl.col("date") >= start_date if start_date else pl.lit(True)
    end_condition = pl.col("date") <= end_date if end_date else pl.lit(True)

    return (
        pl.read_csv(Path(file_path))
        .with_columns(pl.col("date").str.to_date(format="%d/%m/%y"))
        .filter(start_condition & end_condition)
        .sort("date")
    )

# Test
---

In [197]:
df = load_retail_data("../../data/raw/retail/")

In [198]:
df = (
    df.filter(
        pl.col("BRAND_DESCRIPTION").is_in(
            ["BP", "Caltex", "Caltex Woolworths", "Coles Express", "Gull"]
        )
    )
    .group_by(["BRAND_DESCRIPTION", "PUBLISH_DATE"])
    .agg(pl.col("PRODUCT_PRICE").mean().alias("PRICE"))
    .sort("PUBLISH_DATE", descending=False)
)

In [199]:
df_tgp = load_tgp_data("../../data/tgpmin.csv")

In [200]:
df = df.pivot(values="PRICE", index="PUBLISH_DATE", on="BRAND_DESCRIPTION")

In [201]:
# merfe df with df_tgp on PUBLISH_DATE
df = df.join(df_tgp, left_on="PUBLISH_DATE", right_on="date", how="left")
df = df[
    ["PUBLISH_DATE", "BP", "Caltex", "Coles Express", "Caltex Woolworths", "tgpmin"]
]

In [202]:
# add empty columns for shares with null values
for brand in df.columns[1:-1]:  # Exclude 'PUBLISH_DATE'
    df = df.with_columns(pl.lit(None).alias(f"{brand}_share"))

In [203]:
df = df.to_pandas()

In [204]:
df.iloc[0, 6:] = [
    0.32352941176470584,
    0.23529411764705882,
    0.2058823529411765,
    0.23529411764705882,
]

In [205]:
df.iloc[0, :]

PUBLISH_DATE               2009-01-01 00:00:00
BP                                  101.063077
Caltex                               99.387805
Coles Express                        99.691837
Caltex Woolworths                     99.17027
tgpmin                                    95.2
BP_share                              0.323529
Caltex_share                          0.235294
Coles Express_share                   0.205882
Caltex Woolworths_share               0.235294
Name: 0, dtype: object

In [206]:
# shares = [0.22, 0.16, 0.14, 0.16]
# sum_shares = sum(shares)
# shares_norm = [s / sum_shares for s in shares]  # Normalize shares to sum to 1
# print(f"Normalized Shares: {shares_norm}\n")
# # shares_norm = [0.32352941176470584, 0.23529411764705882, 0.2058823529411765, 0.23529411764705882]
# prices = [101.06307692307684, 99.38780487804883, 99.69183673469385, 99.17027027027031]
# alpha = 100
# ci = 95.2 /alpha
# mu = 0.05
# results = simulate_market_with_shares(shares, prices, alpha=alpha, beta=100, mu=mu, a0=0, ci=ci)

# print(f"Monopoly Prices: {results['monopoly_prices']}")
# print(f"Monopoly Market Shares: {results['monopoly_market_shares']}\n")

# print(f"Nash Prices: {results['nash_prices']}")
# print(f"Nash Market Shares: {results['nash_market_shares']}")

In [207]:
df.head(10)

,PUBLISH_DATE,BP,Caltex,Coles Express,Caltex Woolworths,tgpmin,BP_share,Caltex_share,Coles Express_share,Caltex Woolworths_share
0,2009-01-01,101.063077,99.387805,99.691837,99.170270,95.2,0.323529,0.235294,0.205882,0.235294
1,2009-01-02,100.647692,99.153659,99.451020,98.807317,95.2,None,None,None,None
2,2009-01-03,99.890769,98.831707,99.234694,98.463415,95.2,None,None,None,None
3,2009-01-04,99.598462,98.326829,98.471429,98.030769,95.2,None,None,None,None
4,2009-01-05,99.030769,98.124390,97.969388,97.751220,92.5,None,None,None,None
5,2009-01-06,98.561538,97.685366,97.928571,97.300000,93.9,None,None,None,None
6,2009-01-07,100.198462,97.392683,97.716327,97.046341,93.9,None,None,None,None
7,2009-01-08,100.143077,102.263415,97.716327,96.875610,93.9,None,None,None,None
8,2009-01-09,104.081538,102.141463,99.981633,96.773171,95.8,None,None,None,None
9,2009-01-10,105.029231,108.119512,99.981633,99.387805,95.8,None,None,None,None


In [154]:
alpha = 100
mu = 1.5
beta = 100
a0 = 0
for i in range(100):
    idx = i
    row = df.iloc[idx]
    print(f"Processing row {idx}")
    shares = row[6:].values
    prices = [row["BP"], row["Caltex"], row["Coles Express"], row["Caltex Woolworths"]]

    ci = row["tgpmin"] / alpha

    results = simulate_market_with_shares(
        shares, prices, alpha=alpha, beta=beta, mu=mu, a0=a0, ci=ci
    )

    df.loc[idx + 1, "BP_share"] = results["monopoly_market_shares"][0]
    df.loc[idx + 1, "Caltex_share"] = results["monopoly_market_shares"][1]
    df.loc[idx + 1, "Coles Express_share"] = results["monopoly_market_shares"][2]
    df.loc[idx + 1, "Caltex Woolworths_share"] = results["monopoly_market_shares"][3]

Processing row 0
Processing row 1
Processing row 2
Processing row 3
Processing row 4
Processing row 5
Processing row 6
Processing row 7
Processing row 8
Processing row 9
Processing row 10
Processing row 11
Processing row 12
Processing row 13
Processing row 14
Processing row 15
Processing row 16
Processing row 17
Processing row 18
Processing row 19
Processing row 20
Processing row 21
Processing row 22
Processing row 23
Processing row 24
Processing row 25
Processing row 26
Processing row 27
Processing row 28
Processing row 29
Processing row 30
Processing row 31
Processing row 32
Processing row 33
Processing row 34
Processing row 35
Processing row 36
Processing row 37
Processing row 38
Processing row 39
Processing row 40
Processing row 41
Processing row 42
Processing row 43
Processing row 44
Processing row 45
Processing row 46
Processing row 47
Processing row 48
Processing row 49
Processing row 50
Processing row 51
Processing row 52
Processing row 53
Processing row 54
Processing row 55
Pr

In [85]:
df

,PUBLISH_DATE,BP,Caltex,Coles Express,Caltex Woolworths,tgpmin,BP_share,Caltex_share,Coles Express_share,Caltex Woolworths_share
0,2009-01-01,101.063077,99.387805,99.691837,99.170270,95.2,0.323529,0.235294,0.205882,0.235294
1,2009-01-02,100.647692,99.153659,99.451020,98.807317,95.2,0.36078,0.221913,0.200168,0.217138
2,2009-01-03,99.890769,98.831707,99.234694,98.463415,95.2,0.396406,0.209989,0.195129,0.198476
3,2009-01-04,99.598462,98.326829,98.471429,98.030769,95.2,0.421661,0.200921,0.194381,0.183038
4,2009-01-05,99.030769,98.124390,97.969388,97.751220,92.5,0.454018,0.190506,0.186989,0.168487
...,...,...,...,...,...,...,...,...,...,...
2187,2014-12-28,121.831343,119.697101,117.623077,117.300000,109.0,None,None,None,None
2188,2014-12-29,119.189552,118.160870,115.738462,115.550000,109.0,None,None,None,None
2189,2014-12-30,117.816418,114.195652,113.853846,113.762500,110.3,None,None,None,None
2190,2014-12-31,115.353731,112.334783,111.796154,111.637500,110.9,None,None,None,None
